In [1]:
import math
import numpy as np
import pandas as pd
import keras
import tensorflow as tf
import gwpy

Using TensorFlow backend.


In [2]:
from numpy.random import random
from gwpy.timeseries import TimeSeries
from gwpy.frequencyseries import FrequencySeries

In [190]:
# timeSeries = TimeSeries(data["event_time"])
# timeSeries
# gwdata = TimeSeries.get('H1:LDAS-STRAIN', 'September 16 2010 06:40', 'September 16 2010 06:50')
# spec = FrequencySeries([1,2,3,4,5,6,7,8,9,10], f0=0, df=1)
# spec
# spectrum = gwdata.asd(8, 4)

In [174]:
data = pd.read_csv('trainingset_v1d1_metadata.csv') 
features = [ # Selects the features that will go into the model
    "duration",
    "peak_frequency",
    "central_freq",
    "bandwidth",
    "amplitude",
    "snr"] # snr = signalToNoiseRation

In [175]:
filtered = np.array(data[features]) # gets only the selected features for the datapoints
labels = np.array(data["label"]) # gets glitch classification for validation purposes 
labelConvertionDict = createLabelConvertionDict(labels) # dict to convert strLabels to int representations for the model
numLabels = convertStringLabelsToNumeric(labels, labelConvertionDict) # labels are now converted to numerical values

In [176]:
dataSize = len(data) # number of datapoints in the file
trainSize = math.ceil(0.8 * dataSize) # first 80% elements of dataset become training. Remaining 20% is used for validation
inputSize = len(features) # number of features going into the model

In [183]:
trainingSet = filtered[:trainSize] 
trainingLabels = numLabels[:trainSize]
validationSet = filtered[trainSize:]  
validationLabels = numLabels[trainSize:]

In [184]:
model = keras.Sequential() # creates classifier
model.add(keras.layers.Dense(50, input_dim=inputSize, activation='relu'))
model.add(keras.layers.Dense(50, activation='relu'))
model.add(keras.layers.Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')

In [191]:
history = model.fit(trainingSet, trainingLabels, epochs=2048, batch_size=32)

Epoch 1/1024
6373/6373 [==============================] - ETA: 5s - loss: 18.01 - ETA: 0s - loss: 20.85 - ETA: 0s - loss: 21.58 - ETA: 0s - loss: 22.03 - ETA: 0s - loss: 23.15 - ETA: 0s - loss: 22.58 - ETA: 0s - loss: 22.36 - ETA: 0s - loss: 22.31 - ETA: 0s - loss: 23.20 - ETA: 0s - loss: 23.43 - ETA: 0s - loss: 23.39 - 1s 89us/step - loss: 23.3427
Epoch 2/1024
6373/6373 [==============================] - ETA: 0s - loss: 22.59 - ETA: 0s - loss: 24.03 - ETA: 0s - loss: 22.47 - ETA: 0s - loss: 21.82 - ETA: 0s - loss: 21.61 - ETA: 0s - loss: 21.69 - ETA: 0s - loss: 21.35 - ETA: 0s - loss: 21.37 - ETA: 0s - loss: 21.47 - ETA: 0s - loss: 22.06 - ETA: 0s - loss: 22.14 - ETA: 0s - loss: 22.23 - ETA: 0s - loss: 22.36 - 1s 107us/step - loss: 22.4481
Epoch 3/1024
6373/6373 [==============================] - ETA: 0s - loss: 28.68 - ETA: 0s - loss: 21.15 - ETA: 0s - loss: 21.64 - ETA: 0s - loss: 51.37 - ETA: 0s - loss: 93.67 - ETA: 0s - loss: 82.02 - ETA: 0s - loss: 74.54 - ETA: 0s - loss: 67.52 -

Epoch 27/1024
6373/6373 [==============================] - ETA: 0s - loss: 33.99 - ETA: 0s - loss: 23.62 - ETA: 0s - loss: 21.81 - ETA: 0s - loss: 21.55 - ETA: 0s - loss: 22.06 - ETA: 0s - loss: 21.54 - ETA: 0s - loss: 21.17 - 0s 56us/step - loss: 21.8644
Epoch 28/1024
6373/6373 [==============================] - ETA: 0s - loss: 38.47 - ETA: 0s - loss: 21.38 - ETA: 0s - loss: 20.29 - ETA: 0s - loss: 20.52 - ETA: 0s - loss: 20.32 - ETA: 0s - loss: 19.97 - ETA: 0s - loss: 20.08 - 0s 56us/step - loss: 20.4072
Epoch 29/1024
6373/6373 [==============================] - ETA: 0s - loss: 30.31 - ETA: 0s - loss: 28.44 - ETA: 0s - loss: 24.12 - ETA: 0s - loss: 22.27 - ETA: 0s - loss: 29.15 - ETA: 0s - loss: 32.21 - ETA: 0s - loss: 30.72 - 0s 57us/step - loss: 29.4593
Epoch 30/1024
6373/6373 [==============================] - ETA: 0s - loss: 12.82 - ETA: 0s - loss: 274.138 - ETA: 0s - loss: 169.297 - ETA: 0s - loss: 133.436 - ETA: 0s - loss: 108.044 - ETA: 0s - loss: 97.629 - ETA: 0s - loss: 81.6

Epoch 54/1024
6373/6373 [==============================] - ETA: 0s - loss: 30.08 - ETA: 0s - loss: 20.35 - ETA: 0s - loss: 19.28 - ETA: 0s - loss: 18.61 - ETA: 0s - loss: 18.22 - ETA: 0s - loss: 18.07 - ETA: 0s - loss: 17.62 - ETA: 0s - loss: 17.66 - ETA: 0s - loss: 17.86 - 0s 70us/step - loss: 18.0022
Epoch 55/1024
6373/6373 [==============================] - ETA: 0s - loss: 18.71 - ETA: 0s - loss: 20.46 - ETA: 0s - loss: 20.06 - ETA: 0s - loss: 20.70 - ETA: 0s - loss: 19.62 - ETA: 0s - loss: 19.16 - ETA: 0s - loss: 19.02 - ETA: 0s - loss: 18.59 - 0s 63us/step - loss: 18.4871
Epoch 56/1024
6373/6373 [==============================] - ETA: 0s - loss: 17.69 - ETA: 0s - loss: 17.06 - ETA: 0s - loss: 16.44 - ETA: 0s - loss: 16.34 - ETA: 0s - loss: 17.25 - ETA: 0s - loss: 17.25 - ETA: 0s - loss: 17.19 - ETA: 0s - loss: 17.38 - 0s 61us/step - loss: 17.4440
Epoch 57/1024
6373/6373 [==============================] - ETA: 0s - loss: 14.44 - ETA: 0s - loss: 16.17 - ETA: 0s - loss: 15.56 - ETA: 

Epoch 81/1024
6373/6373 [==============================] - ETA: 0s - loss: 13.45 - ETA: 0s - loss: 16.71 - ETA: 0s - loss: 18.13 - ETA: 0s - loss: 17.68 - ETA: 0s - loss: 20.68 - ETA: 0s - loss: 20.38 - ETA: 0s - loss: 19.68 - 0s 53us/step - loss: 19.4559
Epoch 82/1024
6373/6373 [==============================] - ETA: 0s - loss: 8.571 - ETA: 0s - loss: 19.79 - ETA: 0s - loss: 18.22 - ETA: 0s - loss: 17.49 - ETA: 0s - loss: 17.73 - ETA: 0s - loss: 17.43 - ETA: 0s - loss: 17.38 - ETA: 0s - loss: 17.09 - 0s 59us/step - loss: 16.9253
Epoch 83/1024
6373/6373 [==============================] - ETA: 0s - loss: 14.52 - ETA: 0s - loss: 14.30 - ETA: 0s - loss: 14.82 - ETA: 0s - loss: 15.22 - ETA: 0s - loss: 15.07 - ETA: 0s - loss: 15.02 - ETA: 0s - loss: 15.13 - ETA: 0s - loss: 15.30 - ETA: 0s - loss: 15.57 - ETA: 0s - loss: 15.39 - 1s 84us/step - loss: 15.3453
Epoch 84/1024
6373/6373 [==============================] - ETA: 0s - loss: 11.32 - ETA: 0s - loss: 14.09 - ETA: 0s - loss: 14.33 - ETA: 

6373/6373 [==============================] - ETA: 1s - loss: 10.30 - ETA: 0s - loss: 15.22 - ETA: 0s - loss: 15.05 - ETA: 0s - loss: 14.33 - ETA: 0s - loss: 14.10 - ETA: 0s - loss: 14.18 - ETA: 0s - loss: 13.83 - ETA: 0s - loss: 13.61 - ETA: 0s - loss: 13.62 - 0s 74us/step - loss: 13.5881
Epoch 107/1024
6373/6373 [==============================] - ETA: 0s - loss: 20.29 - ETA: 1s - loss: 13.65 - ETA: 0s - loss: 13.07 - ETA: 0s - loss: 13.42 - ETA: 0s - loss: 13.38 - ETA: 0s - loss: 13.85 - ETA: 0s - loss: 14.00 - ETA: 0s - loss: 14.08 - ETA: 0s - loss: 13.98 - ETA: 0s - loss: 14.04 - ETA: 0s - loss: 14.18 - ETA: 0s - loss: 14.20 - ETA: 0s - loss: 13.92 - ETA: 0s - loss: 13.93 - ETA: 0s - loss: 13.98 - ETA: 0s - loss: 13.98 - 1s 139us/step - loss: 13.9477
Epoch 108/1024
6373/6373 [==============================] - ETA: 0s - loss: 16.55 - ETA: 0s - loss: 11.52 - ETA: 0s - loss: 13.13 - ETA: 0s - loss: 14.17 - ETA: 0s - loss: 13.92 - ETA: 0s - loss: 14.07 - ETA: 0s - loss: 13.94 - ETA: 0s 

...

Epoch 892/1024
6373/6373 [==============================] - ETA: 0s - loss: 9.984 - ETA: 0s - loss: 10.72 - ETA: 0s - loss: 11.28 - ETA: 0s - loss: 10.87 - ETA: 0s - loss: 10.99 - ETA: 0s - loss: 10.91 - ETA: 0s - loss: 10.70 - ETA: 0s - loss: 10.56 - ETA: 0s - loss: 10.82 - ETA: 0s - loss: 10.81 - 0s 74us/step - loss: 10.7999
Epoch 893/1024
6373/6373 [==============================] - ETA: 2s - loss: 12.68 - ETA: 0s - loss: 11.66 - ETA: 0s - loss: 11.32 - ETA: 0s - loss: 11.39 - ETA: 0s - loss: 11.21 - ETA: 0s - loss: 11.03 - ETA: 0s - loss: 10.73 - ETA: 0s - loss: 10.76 - 0s 62us/step - loss: 10.7735
Epoch 894/1024
6373/6373 [==============================] - ETA: 0s - loss: 7.177 - ETA: 0s - loss: 12.26 - ETA: 0s - loss: 12.23 - ETA: 0s - loss: 11.13 - ETA: 0s - loss: 11.05 - ETA: 0s - loss: 11.08 - ETA: 0s - loss: 11.09 - ETA: 0s - loss: 10.87 - 0s 60us/step - loss: 10.8471
Epoch 895/1024
6373/6373 [==============================] - ETA: 0s - loss: 11.37 - ETA: 0s - loss: 11.82 - E

6373/6373 [==============================] - ETA: 0s - loss: 3.853 - ETA: 0s - loss: 9.790 - ETA: 0s - loss: 9.965 - ETA: 0s - loss: 9.776 - ETA: 0s - loss: 9.795 - ETA: 0s - loss: 9.978 - ETA: 0s - loss: 10.24 - 0s 56us/step - loss: 10.4448
Epoch 921/1024
6373/6373 [==============================] - ETA: 0s - loss: 9.624 - ETA: 0s - loss: 10.80 - ETA: 0s - loss: 11.03 - ETA: 0s - loss: 10.75 - ETA: 0s - loss: 10.57 - ETA: 0s - loss: 10.82 - ETA: 0s - loss: 10.77 - ETA: 0s - loss: 10.70 - 0s 61us/step - loss: 10.6611
Epoch 922/1024
6373/6373 [==============================] - ETA: 0s - loss: 23.81 - ETA: 0s - loss: 11.28 - ETA: 0s - loss: 10.48 - ETA: 0s - loss: 10.40 - ETA: 0s - loss: 10.49 - ETA: 0s - loss: 10.63 - ETA: 0s - loss: 10.55 - ETA: 0s - loss: 10.71 - 0s 61us/step - loss: 10.7546
Epoch 923/1024
6373/6373 [==============================] - ETA: 0s - loss: 4.920 - ETA: 0s - loss: 9.409 - ETA: 0s - loss: 9.798 - ETA: 0s - loss: 9.946 - ETA: 0s - loss: 9.971 - ETA: 0s - loss: 

6373/6373 [==============================] - ETA: 2s - loss: 6.707 - ETA: 0s - loss: 10.08 - ETA: 0s - loss: 10.53 - ETA: 0s - loss: 10.74 - ETA: 0s - loss: 10.95 - ETA: 0s - loss: 11.62 - ETA: 0s - loss: 11.47 - ETA: 0s - loss: 11.44 - 0s 65us/step - loss: 11.3429
Epoch 949/1024
6373/6373 [==============================] - ETA: 0s - loss: 13.22 - ETA: 0s - loss: 11.29 - ETA: 0s - loss: 11.08 - ETA: 0s - loss: 11.22 - ETA: 0s - loss: 11.29 - ETA: 0s - loss: 11.05 - ETA: 0s - loss: 11.36 - ETA: 0s - loss: 11.30 - 0s 62us/step - loss: 11.3185
Epoch 950/1024
6373/6373 [==============================] - ETA: 0s - loss: 6.349 - ETA: 0s - loss: 12.43 - ETA: 0s - loss: 11.60 - ETA: 0s - loss: 10.78 - ETA: 0s - loss: 10.87 - ETA: 0s - loss: 10.98 - ETA: 0s - loss: 10.94 - ETA: 0s - loss: 11.12 - ETA: 0s - loss: 11.18 - ETA: 0s - loss: 11.26 - 1s 81us/step - loss: 11.3173
Epoch 951/1024
6373/6373 [==============================] - ETA: 1s - loss: 16.25 - ETA: 0s - loss: 10.80 - ETA: 0s - loss: 

6373/6373 [==============================] - ETA: 0s - loss: 8.452 - ETA: 0s - loss: 11.54 - ETA: 0s - loss: 10.58 - ETA: 0s - loss: 10.78 - ETA: 0s - loss: 11.03 - ETA: 0s - loss: 11.23 - ETA: 0s - loss: 11.40 - 0s 55us/step - loss: 11.3480
Epoch 976/1024
6373/6373 [==============================] - ETA: 0s - loss: 6.416 - ETA: 0s - loss: 11.26 - ETA: 0s - loss: 10.99 - ETA: 0s - loss: 11.14 - ETA: 0s - loss: 11.33 - ETA: 0s - loss: 11.36 - ETA: 0s - loss: 11.33 - 0s 54us/step - loss: 11.4194
Epoch 977/1024
6373/6373 [==============================] - ETA: 1s - loss: 4.518 - ETA: 0s - loss: 10.55 - ETA: 0s - loss: 10.72 - ETA: 0s - loss: 10.95 - ETA: 0s - loss: 11.23 - ETA: 0s - loss: 11.34 - ETA: 0s - loss: 11.26 - ETA: 0s - loss: 11.27 - ETA: 0s - loss: 11.43 - ETA: 0s - loss: 34.92 - 0s 77us/step - loss: 33.9913
Epoch 978/1024
6373/6373 [==============================] - ETA: 1s - loss: 16.94 - ETA: 0s - loss: 10.34 - ETA: 0s - loss: 11.11 - ETA: 0s - loss: 11.45 - ETA: 0s - loss: 

6373/6373 [==============================] - ETA: 0s - loss: 13.79 - ETA: 0s - loss: 11.29 - ETA: 0s - loss: 10.69 - ETA: 0s - loss: 10.76 - ETA: 0s - loss: 10.99 - ETA: 0s - loss: 11.12 - ETA: 0s - loss: 11.08 - ETA: 0s - loss: 11.11 - 0s 61us/step - loss: 11.0563
Epoch 1003/1024
6373/6373 [==============================] - ETA: 0s - loss: 9.960 - ETA: 0s - loss: 10.87 - ETA: 0s - loss: 11.23 - ETA: 0s - loss: 11.11 - ETA: 0s - loss: 11.21 - ETA: 0s - loss: 11.42 - ETA: 0s - loss: 11.29 - ETA: 0s - loss: 11.21 - ETA: 0s - loss: 10.93 - ETA: 0s - loss: 11.03 - ETA: 0s - loss: 11.08 - 1s 86us/step - loss: 11.0783
Epoch 1004/1024
6373/6373 [==============================] - ETA: 0s - loss: 4.986 - ETA: 0s - loss: 10.05 - ETA: 0s - loss: 9.5056 - ETA: 0s - loss: 9.787 - ETA: 0s - loss: 10.11 - ETA: 0s - loss: 10.46 - ETA: 0s - loss: 10.30 - ETA: 0s - loss: 10.74 - ETA: 0s - loss: 10.91 - ETA: 0s - loss: 10.95 - ETA: 0s - loss: 11.10 - 1s 86us/step - loss: 11.0234
Epoch 1005/1024
6373/6373

In [197]:
preds = model.predict(validationSet) # uses features from 20% of the original set to try to predict the glitch type
score = score(preds, validationLabels, labelConvertionDict) # compares the predicted glitch to the actual glitch to check accuracy 

Correct: 487
Total: 1593
Accuracy: 30.57%


In [195]:
def score(predictions, validation, labelConvertionDict):
    correct = 0
    count = 0
    # predictions and validation are floats and must be converted to the glitch string label
    predictions = convertNumericLabelsToString(predictions.round(0), labelConvertionDict)
    validation = convertNumericLabelsToString(validation, labelConvertionDict)
    for i in range(len(predictions)):
        #print(predictions[i] + ' <- pred. | actual -> ' + validation[i])
        if predictions[i] == validation[i]:
            correct += 1
        count += 1
    print("Correct: " + str(correct))
    print("Total: " + str(count))
    print("Accuracy: " + str(round((100.0 * correct) / count, 2)) + "%")
    return 100.0 * correct / count

In [112]:
def createLabelConvertionDict(strLabels): # Creates a dict to convert between the string labels to numeric values
    distinctLabels = [elem for elem in strLabels] # deep copy
    labelConvertionDict = dict()
    distinctLabels = list(set(distinctLabels)) # gets just the distinct labels
    numVal = 0
    for label in distinctLabels:
        labelConvertionDict[label] = numVal # assigns each glitch name to a distinct num value
        numVal += 1
    return labelConvertionDict

In [111]:
def convertStringLabelsToNumeric(strLabels, labelConvertionDict): # Converts the labels set to their num values from the dict
    numLabels = [elem for elem in strLabels] # deep copy
    for i in range(len(numLabels)):
        numLabels[i] = labelConvertionDict[numLabels[i]] # replace string label value for corresponding the numValue from dict
    return numLabels

In [110]:
def convertNumericLabelsToString(intLabels, labelConvertionDict):
    converted = [i for i in intLabels] # deep copy
    converted = [i % len(labelConvertionDict) for i in converted] # Prevents prediction to be outside range of num values. Basically takes a random guess if it happens
    strValues = list(labelConvertionDict.keys())
    intToStrDict = dict()
    key = 0
    for val in strValues: # values will be in the same order as the labelConvertionDict so this is a safe operation
        intToStrDict[key] = val
        key += 1
    for i in range(len(converted)):
        converted[i] = str(intToStrDict[int(converted[i])]) # puts in the converted list the stringVal corresponding to the intLabel
    return converted
        